# Tutorial 3: Clipper in Action with Pong

We have already explored some of the features of Clipper (insert recap here). Now let's take a look at Clipper in action with Pong! Released by Atari in 1972, Pong was the first commercially successful video game. You can read more about it <a href="https://en.wikipedia.org/wiki/Pong">here</a>

The goal of this tutorial is to use Clipper to deploy several AI policies to play against, and in doing so explore:
(list of Clipper features we want to highlight)

This tutorial will be broken up into 3 main parts:
#### 1. Starting Clipper and Deploying First Policy
#### 2. Training a Better Model
#### 3. Deploying an updated, trained model

## Part 1: Starting Clipper and Deploying First Policy

Install packaged env from requirements.txt (not sure if we should do this before opening jupyter notebook or not)

In [1]:
#Import dependencies
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as py_deployer
import random
import numpy as np
import pandas as pd
from datascience import *
from sklearn import linear_model
from IPython.display import Image
from IPython.core.display import HTML 


Our first step is to get Clipper started and deploy our first AI policy: one that guesses which direction to move the paddle randomly. First, start Docker (or make sure it is running)

In [2]:
#Starting Clipper, make sure you have Docker running before you run this cell
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

18-09-05:22:04:23 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
18-09-05:22:04:23 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
18-09-05:22:04:26 INFO     [clipper_admin.py:126] Clipper is running


Running the cell below will register an application in Clipper called "pong" and create a Clipper endpoint for our random policy at http://localhost:1337/pong/predict

In [3]:
def random_predict(xs):
    action = random.randint(0, 2)
    return [str(action) for _ in xs]

py_deployer.create_endpoint(clipper_conn, name="pong", input_type="doubles", func=random_predict,
                            default_output="0", slo_micros=100000)

18-09-05:22:04:58 INFO     [clipper_admin.py:201] Application pong was successfully registered
18-09-05:22:04:58 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmpxesfqfdo
18-09-05:22:04:58 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-09-05:22:04:58 INFO     [python.py:192] Python closure saved
18-09-05:22:04:58 INFO     [python.py:202] Using Python 3.5 base image
18-09-05:22:04:58 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmpxesfqfdo
18-09-05:22:04:59 INFO     [clipper_admin.py:456] {'stream': 'Step 1/2 : FROM clipper/python35-closure-container:0.3.0'}
18-09-05:22:04:59 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05:22:04:59 INFO     [clipper_admin.py:456] {'stream': ' ---> 1fc910c1ed28\n'}
18-09-05:22:04:59 INFO     [clipper_admin.py:456] {'stream': 'Step 2/2 : COPY /tmp/clipper/tmpxesfqfdo /model/'}
18-09-05:22:04:59 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05

Now that we have a model deployed, let's see how it works! Run the cell below to start the game

In [4]:
!python pong-server.py localhost:1337

18-09-05:22:05:14 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
18-09-05:22:05:15 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/index.html
127.0.0.1 - - [05/Sep/2018 22:05:15] "GET /pong/ HTTP/1.1" 200 -
18-09-05:22:05:15 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/pong.css
127.0.0.1 - - [05/Sep/2018 22:05:15] "GET /pong/pong.css HTTP/1.1" 200 -
18-09-05:22:05:15 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/game.js
127.0.0.1 - - [05/Sep/2018 22:05:15] "GET /pong/game.js HTTP/1.1" 200 -
18-09-05:22:05:15 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/pong.js
127.0.0.1 - - [05/Sep/2018 22:05:15] "GET /pong/pong.js HTTP/1.1" 200 -
18-09-05:22:05:15 INFO     [pong-server.py:51] Local path: /Users/hari/Desktop/Cal/rise/clipper-tutorials/static/images/press1.png
18-09-05:22:

Congratulations! We have depolyed our first model to Clipper! Let's take a look at it by clicking http://localhost:3000/pong/ and pressing 1 to start the game.

## Part 2: Training a better model

As you probably noticed, the random-guessing policy did not perform well at all. In order to train a better model, we are going to use imitation learning. Let's play the game again, this time with 2 actual players, and we will collect data on how you play the game to train our model! 

Go to http://localhost:3000/pong/ and press 2. Find someone next to you and play pong against them!!

<b>Part to collect data into a output.csv --> currently doing this in really hacky way to just get it to work, but trying to figure out better solution</b>

Now that we have our data, lets train a new model! First, run the cell below to clean the data and format it for scikit-learn's LogisticRegression model.

In [5]:
#Read data into table
data_table= Table.read_table('out.csv')

#Convert labels into numeric values
def convert_label(label):
    if(label=="down"):
        return 1
    elif(label=="up"):
        return 2
    else:
        return 0
data_table_2 = data_table.with_column("label", data_table.apply(convert_label, 0))

#Scale our data down
final_table = Table().with_columns("label", data_table.apply(convert_label, 0),
                                  "paddle_y", data_table.column(1)/500.0,
                                  "ball_x", data_table.column(2)/500.0,
                                  "ball_y", data_table.column(3)/500.0,
                                  "ball_dx", data_table.column(4)/500.0,
                                  "ball_dy", data_table.column(5)/500.0,
                                  "x_prev", data_table.column(6)/500.0,
                                  "y_prev", data_table.column(7)/500.0)
data_df = final_table.to_df()
data_df.take(np.arange(5))

,label,paddle_y,ball_x,ball_y,ball_dx,ball_dy,x_prev,y_prev
0,1,0.6576,0.416920,0.606838,0.335032,0.335032,0.400188,0.590107
1,1,0.6774,0.433692,0.623610,0.335832,0.335832,0.416920,0.606838
2,1,0.7170,0.467355,0.657273,0.337432,0.337432,0.449157,0.639075
3,1,0.7566,0.501178,0.691096,0.339032,0.339032,0.484246,0.674165
4,1,0.7764,0.518150,0.708068,0.339832,0.339832,0.501178,0.691096


We are going to use the data to train a scikit-learn Logistic Regression model. You can read more about the particular model <a url="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">here</a>

In [6]:
labels = data_df['label']
training_data= data_df.drop(['label'], axis=1)

model = linear_model.LogisticRegression()
model.fit(training_data, labels)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Part 3: Deploying updated model

Now that we have an updated model, we can deploy that model onto Clipper. Once the system realizes there is a new version of the model, it will automatically switch to serving the newer version of the model.

In [7]:
Image(url= "update_model.png")

In [8]:
def predict(inputs):
    # model.predict returns a list of predictions
    preds = model.predict(inputs)
    return [str(p) for p in preds]

py_deployer.deploy_python_closure(clipper_conn, name="pong", version=3, input_type="doubles", func=predict, pkgs_to_install=["numpy","scipy", "pandas", "datascience", "sklearn"])


18-09-05:22:09:03 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmp7_o4vxso
18-09-05:22:09:03 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-09-05:22:09:03 INFO     [python.py:192] Python closure saved
18-09-05:22:09:03 INFO     [python.py:202] Using Python 3.5 base image
18-09-05:22:09:03 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmp7_o4vxso
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Step 1/3 : FROM clipper/python35-closure-container:0.3.0'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': ' ---> 1fc910c1ed28\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Step 2/3 : COPY /tmp/clipper/tmp7_o4vxso /model/'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': ' ---> de1022770b73\n'}
18-09-05:22:09:52 

18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Collecting snowballstemmer>=1.1 (from sphinx->datascience)\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/d4/6c/8a935e2c7b54a37714656d753e4187ee0631988184ed50c0cf6476858566/snowballstemmer-1.2.1-py2.py3-none-any.whl (64kB)\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Collecting packaging (from sphinx->datascience)\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/ad/c2/b500ea05d5f9f361a562f089fc91f77ed3b4783e13a08a3daf82069b1224/packaging-17.1-py2.py3-none-any.whl\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Collecting babel!=2.0,>=1.3 (from sphinx->datascience)\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': '  Downloading https://files.pythonhosted.org/packages/b8/ad/c6f60602d3ee3d92fbed87675b6fb6a6f9a38c223343ababdb44ba201f

18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': "\x1bYou are using pip version 9.0.3, however version 18.0 is available.\nYou should consider upgrading via the 'pip install --upgrade pip' command.\n\x1b"}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': ' ---> 37f52cefca5f\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'aux': {'ID': 'sha256:37f52cefca5fb55a7660b2fdbb34930224a875bf462b4c71b96db1a5e278125e'}}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Successfully built 37f52cefca5f\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:456] {'stream': 'Successfully tagged pong:3\n'}
18-09-05:22:09:52 INFO     [clipper_admin.py:458] Pushing model Docker image to pong:3
18-09-05:22:09:53 INFO     [docker_container_manager.py:257] Found 0 replicas for pong:3. Adding 1
18-09-05:22:10:00 INFO     [clipper_admin.py:635] Successfully registered model pong:3
18-09-05:22:10:00 INFO     [clipper_admin.py:553] Done deploying model pong:3.


You may notice that it seems like we only deployed the predict function to Clipper. However, Clipper will track the dependencies that the model in the predict function uses and include them in the model Docker container as dependencies. You can take a look at the logs above to see the Docker container downloading said dependencies.

Let's run our server again (or refresh the page if you haven't stopped the server previously)

In [10]:
!python pong-server.py localhost:1337

18-09-05:22:12:10 INFO     [pong-server.py:109] Starting Pong Server on localhost:3000
^C
Traceback (most recent call last):
  File "pong-server.py", line 117, in <module>
    run(clipper_addr)
  File "pong-server.py", line 112, in run
    server.serve_forever()
  File "/Users/hari/anaconda/lib/python3.5/socketserver.py", line 232, in serve_forever
    ready = selector.select(poll_interval)
  File "/Users/hari/anaconda/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt


Cool points here:
1. We never restarted the pong server, but our code seamlessly switches to serving the updated model

## 4. Conclusion

Just as a recap, here's what we did today:

1. Deployed an initial random policy to Clipper and served predictions
2. Trained a new model with data that you generated while playing pong
3. Deployed new version of model and seamlessly switch the model being served

By doing so, we've explored the following Clipper features:

1. Deploy models trained in your choice of framework to Clipper with a few lines of code by using an existing model container or writing your own
2. Easily update or roll back models in running applications
3. Run each model in a separate Docker container for simple cluster management and resource allocation


In [11]:
!python stop_c.py


18-09-05:22:13:12 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers


# TO-DOS


1. Fix copy on tutorial
2. Package dependencies